<a href="https://colab.research.google.com/github/nntadotzip/vi-augmentation/blob/main/bert_pairClassification_SYN_EXP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Install

**Models**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install datasets==1.0.1 
!pip install transformers==3.1.0

     |████████████████████████████████| 1.8 MB 15.3 MB/s 
     |████████████████████████████████| 212 kB 51.5 MB/s 
     |████████████████████████████████| 884 kB 29.4 MB/s 
     |████████████████████████████████| 3.0 MB 62.4 MB/s 
     |████████████████████████████████| 1.2 MB 58.1 MB/s 
     |████████████████████████████████| 895 kB 62.5 MB/s 


In [3]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric

os.environ["TOKENIZERS_PARALLELISM"] = "false"

PyTorch version 1.10.0+cu111 available.
TensorFlow version 2.8.0 available.


In [4]:
# Check that we are using 100% of GPU memory footprint support libraries/code
# from https://github.com/patrickvonplaten/notebooks/blob/master/PyTorch_Reformer.ipynb
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip -q install gputil
!pip -q install psutil
!pip -q install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.9 GB  | Proc size: 1.6 GB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total 15109MB


**Cosine Similarity**

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [6]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Load the Dataset

In [7]:
# from google.colab import files
# files.upload()

In [8]:
original_df_sem0 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/prep/vietjackGeo/labeled_1to1999/labeled_1to2000_csvformat.csv', encoding='utf8')
original_df_sem0.head(2)

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"“dân cư và lao động”|dân số đông, nguồn lao động dồi dào và có trình độ.|nguồn lao động lớn nhất cả nước.|lao động có trình độ cao nhất cả nước.|lao động tập trung chủ yếu ở các thành phố lớn.","Biện pháp cơ bản để đưa đồng bằng sông Hồng sớm trở thành vùng sản xuất lương thực, thực phầm hàng hóa là:|quan tâm đến chất lương sản phẩm và thị trường.|thay đổi cơ cấu cây cây trồng và cơ cấu mùa vụ.|chú ý đến môi trường và bảo vệ tài nguyên đất.|phát triển mạnh cây vụ đông.","dân số đông, nguồn lao động dồi dào và có trình độ.",thay đổi cơ cấu cây cây trồng và cơ cấu mùa vụ.,"0,71565396",0.0
1,"“dân cư và lao động”|dân số đông, nguồn lao động dồi dào và có trình độ.|nguồn lao động lớn nhất cả nước.|lao động có trình độ cao nhất cả nước.|lao động tập trung chủ yếu ở các thành phố lớn.","Tại sao việc làm là một trong những vấn đề nan giải ở Đồng bằng sông Hồng nhất là ở khu vực thành thị?|Do dân nhập cư đông.|Do dân số đông, kết cấu dân số trẻ.|Do nền kinh tế còn chậm phát triển.|Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm","dân số đông, nguồn lao động dồi dào và có trình độ.","Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm","0,8153097",1.0


In [10]:
augmented_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/prep/vietjackGeo/final_augment/syn_augmented_1to1999.csv', encoding='utf8')
augmented_df.head(2)

,sentence1,sentence2,ans_question_1,ans_question_2,manual_label,augmented1,augmented2
0,"“dân cư và lao động”|dân số đông, nguồn lao động dồi dào và có trình độ.|nguồn lao động lớn nhất cả nước.|lao động có trình độ cao nhất cả nước.|lao động tập trung chủ yếu ở các thành phố lớn.","Tại sao việc làm là một trong những vấn đề nan giải ở Đồng bằng sông Hồng nhất là ở khu vực thành thị?|Do dân nhập cư đông.|Do dân số đông, kết cấu dân số trẻ.|Do nền kinh tế còn chậm phát triển.|Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm","dân số đông, nguồn lao động dồi dào và có trình độ.","Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm",1.0,"“ dân cư và lao động ” | dân cư đông , nguồn lao động dồi dào và có trình độ .| nguồn lao động lớn nhất cả nước .| lao động có trình độ cao nhất cả nước .| lao động tập trung chủ yếu ở các thành phố lớn .","tại sao việc làm là một trong những vấn đề nan giải ở đồng bằng sông hồng nhất là ở khu vực thành thị ?| do dân nhập cư đông .| do dân cư đông , kết cấu dân cư trẻ .| do nền kinh tế còn chậm phát triển .| do dân cư đông , kết cấu dân cư trẻ trong điều kiện kinh tế chậm"
1,"Bão, lũ lụt, hạn hán, gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng|Đồng bằng sông Hồng.|Tây Bắc.|Duyên hải miền Trung.|Tây Nguyên",Vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió Tây khô nóng?|Bắc Trung Bộ.|Đông Bắc.|Đông Nam Bộ.|Tây Nguyên.,Duyên hải miền Trung.,Bắc Trung Bộ.,1.0,"bão , lũ lụt , hạn hán , gió mạnh tây khô nóng là thiên tai xảy ra chủ yếu ở vùng | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió mạnh tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| tây nguyên .


In [11]:
augmented_df =  augmented_df[['augmented1', 'augmented2', 'ans_question_1', 'ans_question_2', 'manual_label']]
augmented_df['cosine_similarity'] = ''
augmented_df  = augmented_df[['augmented1', 'augmented2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label']]
augmented_df = augmented_df.set_axis(['question_1', 'question_2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label'], axis=1, inplace=False)
augmented_df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"“ dân cư và lao động ” | dân cư đông , nguồn lao động dồi dào và có trình độ .| nguồn lao động lớn nhất cả nước .| lao động có trình độ cao nhất cả nước .| lao động tập trung chủ yếu ở các thành phố lớn .","tại sao việc làm là một trong những vấn đề nan giải ở đồng bằng sông hồng nhất là ở khu vực thành thị ?| do dân nhập cư đông .| do dân cư đông , kết cấu dân cư trẻ .| do nền kinh tế còn chậm phát triển .| do dân cư đông , kết cấu dân cư trẻ trong điều kiện kinh tế chậm","dân số đông, nguồn lao động dồi dào và có trình độ.","Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm",,1.0
1,"bão , lũ lụt , hạn hán , gió mạnh tây khô nóng là thiên tai xảy ra chủ yếu ở vùng | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió mạnh tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| tây nguyên .,Duyên hải miền Trung.,Bắc Trung Bộ.,,1.0


In [12]:
merged_df = original_df_sem0
merged_df = merged_df.append(augmented_df, sort=False, ignore_index=True)

In [13]:
merged_df.tail(2)

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
10624,"việc phát ra kinh tế - xã hội vùng dân tộc ít người ở nước ta cần được chú trọng hơn nữa do | các dân tộc ít người đóng vai trò rất quan trọng trong việc đảm bảo an ninh quốc phòng .| một số dân tộc ít người có những kinh nghiệm sản xuất quí báu .| sự phát ra kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể , mức sống của bộ phận dân tộc ít người thấp .| trước đây chúng ta chưa chú trọng vấn đề này .","đặc điểm nào không đúng với dân cư , dân tộc ở nước ta ?| các dân tộc luôn phát huy truyền thống sản xuất .| các dân tộc luôn đoàn kết bên nhau .| chất lượng đời sống của các dân tộc ít người đã ở mức cao .| sự phát triển kinh tế - xã hội giữa các vùng còn chênh lệch .","sự phát triển kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể, mức sống của bộ phận dân tộc ít người thấp.",Chất lượng đời sống của các dân tộc ít người đã ở mức cao.,,1.0
10625,"việc phát ra kinh tế - xã hội vùng dân tộc ít người ở nước ta cần được chú trọng hơn nữa do | các dân tộc ít người đóng vai trò rất quan trọng trong việc đảm bảo an ninh quốc phòng .| một số dân tộc ít người có những kinh nghiệm sản xuất quí báu .| sự phát ra kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể , mức sống của bộ phận dân tộc ít người thấp .| trước đây chúng ta chưa chú trọng vấn đề này .","đặc điểm nào không đúng với dân cư , dân tộc ở nước ta ?| các dân tộc luôn phát huy truyền thống sản xuất .| các dân tộc luôn đoàn kết bên nhau .| chất lượng đời sống của các dân tộc ít người đã ở mức cao .| sự phát ra kinh tế - xã hội giữa các vùng còn chênh lệch .","sự phát triển kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể, mức sống của bộ phận dân tộc ít người thấp.",Chất lượng đời sống của các dân tộc ít người đã ở mức cao.,,1.0


In [20]:
merged_df = merged_df.sample(frac = 1)
merged_df = merged_df.reset_index(drop=True)
merged_df

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"căn cứ vào atlat địa lí việt nam trang 9 , hãy cho biết địa điểm nào sau đây có lượng mưa bão trung bình năm trên 2800 mm / năm ?| lạng sơn .| hà nội .| thừa thiên – huế .| tp .hồ chí minh .",đặt nằm nào dưới đây có lượng mưa trung bình năm lớn nhât ?| hà nội .| huế .| nha trang .| phan thiết .,Thừa Thiên – Huế.,Huế.,,1.0
1,"ở miền khí hậu phía bắc , trong mùa đông độ lạnh giảm dần về phía tây vì : | nhiệt độ tăng dần theo vĩ độ .| nhiệt độ thay đổi theo độ cao của địa hình .| đó là những vùng không chịu ảnh hưởng của gió mùa đông bắc .| dãy hoàng liên sơn dầu ngăn cản ảnh hưởng của gió mùa đông bắc .",do tác động của dãy hoàng liên sơn nên khu vực tây bắc có | mùa đông ướp lạnh hơn vùng đông bắc .| chịu ảnh hưởng mạnh của gió tây khô nóng .| có mùa đông ngắn và đỡ ướp lạnh hơn đông bắc .| có mùa đông kéo dài và mùa hạ mát .,Dãy Hoàng Liên Sơn ngăn cản ảnh hưởng của gió mùa đông bắc.,có mùa đông ngắn và đỡ lạnh hơn Đông Bắc.,,1.0
2,"Thuận lợi chủ yếu cho việc khai thác thủy sản ở nước ta là có|nhiều sông suối, kênh rạch, ao hồ.|nhiều cánh rừng ngập mặn.|4 ngư trường trọng điểm.|các ô trũng ở giữa đồng bằng.","Nền nông nghiệp nước ta đang trong quá trình chuyển đổi sâu sắc, với tính chất sản xuất hàng hóa ngày càng cao, quy mô sản xuất ngày càng lớn, nên chịu sự tác động mạnh mẽ của|sự biến động của thị trường.|nguồn lao động đang giảm.|các thiên tai ngày càng tăng.|tính chất bấp bênh vốn có của nông nghiệp.",4 ngư trường trọng điểm.,sự biến động của thị trường.,"0,73334247",0.0
3,"tác động chủ yếu của việc hình thành các vùng chuyên canh cây công nghiệp ở tây nguyên đối với phát triển kinh tế - xã hội của vùng là : | ngăn chặn nạn chặt phá rừng , đốt rừng .| tạo ra tập quán sản xuất mới cho đồng bào các dân tộc tây nguyên .| thu hút hàng vạn hoạt động từ các vùng khác của đất nước về tây nguyên .| bảo vệ đất , hạn chế xói mòn sạt lở .",công nghiệp chế biến của tây nguyên đang được đẩy mạnh chủ yếu là nhờ | điều kiện tự nhiên và tài nguyên thiên nhiên thuận lợi | việc tăng cường nguồn lao động cả về số lượng và chất lượng .| việc tăng cường xây dựng cơ sở hạ tầng và mở rộng thị trường .| nền nông nghiệp hàng hoá của vùng phát ra mạnh .,tạo ra tập quán sản xuất mới cho đồng bào các dân tộc Tây Nguyên.,việc tăng cường xây dựng cơ sở hạ tầng và mở rộng thị trường.,,1.0
4,"phát biểu nào sau đây không đúng với vị trí địa lí , lãnh thổ của trung du và miền núi bắc bộ ?| tiếp giáp hai quốc gia , hai vùng kinh tế .| vị trí thuận lợi cho giao dịch với bên ngoài qua các cửa khẩu .| tài nguyên thiên nhiên giàu có , đa dạng | có diện tích lớn nhất nước ta , nhưng mật độ dân cư không cao .",cơ sở chủ yếu để cơ cấu công nghiệp của trung du và miền núi bắc bộ bao gồm nhiều ngành là : | chính sách phát triển công nghiệp ở miền núi của nhà nước .| tài nguyên thiên nhiên đa dạng hóa .| sự giao lưu thuận lợi với các vùng khác ở trong và ngoài nước .| nguồn lao động có nhiều kinh nghiệm sản xuất truyền thống .,"Tài nguyên thiên nhiên giàu có, đa dạng",tài nguyên thiên nhiên đa dạng.,,1.0
...,...,...,...,...,...,...
10621,"ở miền bắc , đoàn tụ nhiệt đới gió mùa có độ cao trung bình dưới ( m ) : | 400 – 500 .| 500 – 600 .| 600 – 700 .| 700 – 800 .",quân đoàn ôn đới gió mùa trên núi ở độ cao trên ( m ) : | 2.500 .| 2.600 .| 2.700 .| 2.800 .,600 – 700.,2.600.,,1.0
10622,"Căn cứ vào Atlat Địa lí Việt Nam trang 6 -7, hãy cho biết đồng bằng Nghệ An được hình thành do phù sa của sông nào bồi đắp?|sông Mã – Chu.|sông Cả.|sông Gianh.|sông Thu Bồn.","Những yếu tố nào của địa hình đồi núi tác động tạo nên sự phân hóa tự nhiên nước ta?|độ cao và hướng các dãy núi.|độ cao, độ dốc và hướng các dãy núi.|độ dốc và hướng các dãy núi.|độ cao và độ dốc của các dãy núi.",sông Cả.,độ cao và hướng các dãy núi.,"0,76883435",0.0
10623,"căn cứ vào atlat địa lí việt nam trang 23 , cho biết cửa khẩu nào sau đây nằm trên con đường biên giới việt nam – lào ?| móng cái .| lệ thanh .| mường kh

In [21]:
dataset = merged_df.set_axis(['sentence1', 'sentence2', 'ans_sentence1', 'ans_sentence2', 'similarity', 'label'], axis=1, inplace=False)

In [22]:
dataset

,sentence1,sentence2,ans_sentence1,ans_sentence2,similarity,label
0,"căn cứ vào atlat địa lí việt nam trang 9 , hãy cho biết địa điểm nào sau đây có lượng mưa bão trung bình năm trên 2800 mm / năm ?| lạng sơn .| hà nội .| thừa thiên – huế .| tp .hồ chí minh .",đặt nằm nào dưới đây có lượng mưa trung bình năm lớn nhât ?| hà nội .| huế .| nha trang .| phan thiết .,Thừa Thiên – Huế.,Huế.,,1.0
1,"ở miền khí hậu phía bắc , trong mùa đông độ lạnh giảm dần về phía tây vì : | nhiệt độ tăng dần theo vĩ độ .| nhiệt độ thay đổi theo độ cao của địa hình .| đó là những vùng không chịu ảnh hưởng của gió mùa đông bắc .| dãy hoàng liên sơn dầu ngăn cản ảnh hưởng của gió mùa đông bắc .",do tác động của dãy hoàng liên sơn nên khu vực tây bắc có | mùa đông ướp lạnh hơn vùng đông bắc .| chịu ảnh hưởng mạnh của gió tây khô nóng .| có mùa đông ngắn và đỡ ướp lạnh hơn đông bắc .| có mùa đông kéo dài và mùa hạ mát .,Dãy Hoàng Liên Sơn ngăn cản ảnh hưởng của gió mùa đông bắc.,có mùa đông ngắn và đỡ lạnh hơn Đông Bắc.,,1.0
2,"Thuận lợi chủ yếu cho việc khai thác thủy sản ở nước ta là có|nhiều sông suối, kênh rạch, ao hồ.|nhiều cánh rừng ngập mặn.|4 ngư trường trọng điểm.|các ô trũng ở giữa đồng bằng.","Nền nông nghiệp nước ta đang trong quá trình chuyển đổi sâu sắc, với tính chất sản xuất hàng hóa ngày càng cao, quy mô sản xuất ngày càng lớn, nên chịu sự tác động mạnh mẽ của|sự biến động của thị trường.|nguồn lao động đang giảm.|các thiên tai ngày càng tăng.|tính chất bấp bênh vốn có của nông nghiệp.",4 ngư trường trọng điểm.,sự biến động của thị trường.,"0,73334247",0.0
3,"tác động chủ yếu của việc hình thành các vùng chuyên canh cây công nghiệp ở tây nguyên đối với phát triển kinh tế - xã hội của vùng là : | ngăn chặn nạn chặt phá rừng , đốt rừng .| tạo ra tập quán sản xuất mới cho đồng bào các dân tộc tây nguyên .| thu hút hàng vạn hoạt động từ các vùng khác của đất nước về tây nguyên .| bảo vệ đất , hạn chế xói mòn sạt lở .",công nghiệp chế biến của tây nguyên đang được đẩy mạnh chủ yếu là nhờ | điều kiện tự nhiên và tài nguyên thiên nhiên thuận lợi | việc tăng cường nguồn lao động cả về số lượng và chất lượng .| việc tăng cường xây dựng cơ sở hạ tầng và mở rộng thị trường .| nền nông nghiệp hàng hoá của vùng phát ra mạnh .,tạo ra tập quán sản xuất mới cho đồng bào các dân tộc Tây Nguyên.,việc tăng cường xây dựng cơ sở hạ tầng và mở rộng thị trường.,,1.0
4,"phát biểu nào sau đây không đúng với vị trí địa lí , lãnh thổ của trung du và miền núi bắc bộ ?| tiếp giáp hai quốc gia , hai vùng kinh tế .| vị trí thuận lợi cho giao dịch với bên ngoài qua các cửa khẩu .| tài nguyên thiên nhiên giàu có , đa dạng | có diện tích lớn nhất nước ta , nhưng mật độ dân cư không cao .",cơ sở chủ yếu để cơ cấu công nghiệp của trung du và miền núi bắc bộ bao gồm nhiều ngành là : | chính sách phát triển công nghiệp ở miền núi của nhà nước .| tài nguyên thiên nhiên đa dạng hóa .| sự giao lưu thuận lợi với các vùng khác ở trong và ngoài nước .| nguồn lao động có nhiều kinh nghiệm sản xuất truyền thống .,"Tài nguyên thiên nhiên giàu có, đa dạng",tài nguyên thiên nhiên đa dạng.,,1.0
...,...,...,...,...,...,...
10621,"ở miền bắc , đoàn tụ nhiệt đới gió mùa có độ cao trung bình dưới ( m ) : | 400 – 500 .| 500 – 600 .| 600 – 700 .| 700 – 800 .",quân đoàn ôn đới gió mùa trên núi ở độ cao trên ( m ) : | 2.500 .| 2.600 .| 2.700 .| 2.800 .,600 – 700.,2.600.,,1.0
10622,"Căn cứ vào Atlat Địa lí Việt Nam trang 6 -7, hãy cho biết đồng bằng Nghệ An được hình thành do phù sa của sông nào bồi đắp?|sông Mã – Chu.|sông Cả.|sông Gianh.|sông Thu Bồn.","Những yếu tố nào của địa hình đồi núi tác động tạo nên sự phân hóa tự nhiên nước ta?|độ cao và hướng các dãy núi.|độ cao, độ dốc và hướng các dãy núi.|độ dốc và hướng các dãy núi.|độ cao và độ dốc của các dãy núi.",sông Cả.,độ cao và hướng các dãy núi.,"0,76883435",0.0
10623,"căn cứ vào atlat địa lí việt nam trang 23 , cho biết cửa khẩu nào sau đây nằm trên con đường biên giới việt nam – lào ?| móng cái .| lệ thanh .| mường khương .| cầu treo .

**Train Test Split**

In [23]:
!pip install sklearn
from sklearn.model_selection import train_test_split

In [24]:
df_train, df_test = train_test_split(dataset, test_size=0.2, random_state = 8, shuffle=True)
df_train, df_val = train_test_split(dataset, test_size=0.25, random_state = 8)

In [25]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(7969, 6)
(2657, 6)
(2126, 6)


In [26]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [27]:
df_train

,sentence1,sentence2,ans_sentence1,ans_sentence2,similarity,label
0,"Nguyên nhân quan trọng nhất về mặt tự nhiên để cây cao su phát triển mạnh ở Đông Nam Bộ là:|nguồn nước mặt phong phú.|có nhiều cơ sở chế biến mủ cao su trong vùng.|thị trường tiêu thụ cao su rộng lớn và ổn định.|có loại đất xám rất thích hợp cho cây cao su, khí hậu nóng ẩm, ít bão.","Về nông nghiệp, Đông Nam Bộ là vùng|chuyên canh cây lương thực hàng đầu của nước ta.|chuyên canh cây công nghiệp hàng đầu của nước ta.|chuyên canh cây thực phẩm hàng đầu của nước ta.|chăn nuôi gia súc hàng đầu của nước ta.","có loại đất xám rất thích hợp cho cây cao su, khí hậu nóng ẩm, ít bão.",chuyên canh cây công nghiệp hàng đầu của nước ta.,"0,7290428",1.0
1,"ở nước ta thời tiết mùa đông bớt lạnh khô , mùa hè bớt nóng bức là nhờ : | nằm gần xích đạo , mưa nhiều .| địa hình 60% là đồi núi thấp .| chịu tác động thường xuyên của gió mùa .| tiếp giáp nhau với biển đông ( trên 3260 km bờ biển ) .","khí hậu nước ta mang đặc tính khí hậu hải dương , điều hoà hơn là nhờ : | nằm gần xích đạo , mưa nhiều .| địa hình 85% là đầy đủ thấp .| chịu tác động thường xuyên của gió mùa .| tiếp giáp với biển đông .",Tiếp giáp với Biển Đông (trên 3260 km bờ biển).,Tiếp giáp với Biển Đông.,,1.0
2,"căn cứ vào atlat địa lí việt nam trang 9 , hãy cho phép biết ở nước ta tần suất hoạt động của bão nhiều nhất vào tháng nào trong năm ?| tháng 10 .| tháng 11 .| tháng 9 .| tháng 7 .",nguyên nhân chủ yếu làm cho mùa bạo lực nước ta chậm dần từ bắc vào nam là | hình dạng lãnh thổ hẹp ngang và kéo dài theo chiều bắc - nam .| gió mùa đông bắc suy dần khi di chuyển xuống phía nam .| dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bạo lực .| nước ta tiếp giáp với biển đông rộng lớn .,Tháng 9.,dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão.,,1.0
3,"hướng giải quyết vấn đề lương thực , thực phẩm nào sau đây ở bắc trung bộ là chưa hợp lí | phát ra mạnh khai thác và nuôi trồng thuỷ sản để bổ sung thêm lượng đạm trong bữa ăn .| tăng cường thuỷ lợi , đẩy mạnh thâm canh lúa để tăng năng suất và sản lượng lương thực .| phát ra giao thông vận tải đảm bảo việc chuyên chở , trao đổi sản phẩm với các vùng khác .| đẩy mạnh thâm canh lúa , xác định cơ cấu mùa vụ thích hợp để đảm bảo sản xuất và tránh được thiên tai .",giao thông vận tải có vai trò quan trọng đối với sự phát triển kinh tế - xã hội của bắc trung bộ do | có nhiều tuyến đường nối các cảng biển của việt nam với lào .| có nhiều tuyến giao thông quan trọng đi qua .| là địa bàn trung chuyển hàng hoá bắc – nam và đông – tây .| nằm trên con đường xuyên á kết thúc ở các cảng biển của việt nam .,"Phát triển giao thông vận tải đảm bảo việc chuyên chở, trao đổi sản phẩm với các vùng khác.",là địa bàn trung chuyển hàng hóa Bắc – Nam và Đông – Tây.,,1.0
4,"hạn chế nào không phải do hình dạng dài và hẹp của lãnh thổ việt nam mang lại : | khoáng sản nước ta đa dạng , nhưng trữ lượng không lớn | giao thông bắc – nam trắc trở | việc bảo vệ an ninh và chủ quyền lãnh thổ khó khăn | khí hậu phân ủ phức tạp",nước ta có hình dạng lãnh thổ kéo dài và hẹp ngang nên : | nước ta giàu có về tài nguyên khoáng sản | nước ta giàu có về tài nguyên thuỷ sản | khí hậu có sự thụ phấn bắc – nam | thuận lợi cho giao thông vận tải phát triển,"Khoáng sản nước ta đa dạng, nhưng trữ lượng không lớn",khí hậu có sự phân hóa Bắc – Nam,,1.0
...,...,...,...,...,...,...
7964,"đặc điểm nào không đúng với dân cư , dân tộc ở nước ta ?| các dân tộc luôn phát huy truyền thống sản xuất .| các dân tộc luôn đoàn kết bên nhau .| chất lượng đời sống của các dân tộc ít người đã ở mức cao .| sự phát ra kinh tế - xã hội giữa các vùng còn chênh lệch .",đẩy mạnh phát triển công nghiệp ở trung du và miền núi nhằm : | giảm tỉ lệ gia tăng dân số ở khu vực này .| phân bố lại dân cư và lao động giữa các vùng ngoại ô .| tăng tỉ lệ dân thành thị trong cơ cấu dân số .| phát huy truyền thống sản xuất của các dân tộc ít người .,Chất lượng đời sống của các dân tộc ít người đã ở mức cao.,Phân bố lại dân

# Load Bert Model

In [28]:
from transformers import BertTokenizer, BertModel

Tokenize for our dataset using BertTokenizer

In [29]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model='bert-base-cased'):

        self.data = data  # pandas dataframe
        #Initialize the tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(bert_model)  

        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent1 = str(self.data.loc[index, 'sentence1'])
        sent2 = str(self.data.loc[index, 'sentence2'])

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.maxlen,  
                                      return_tensors='pt')  # Return torch.Tensor objects
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.data.loc[index, 'label']
            return token_ids, attn_masks, token_type_ids, label  
        else:
            return token_ids, attn_masks, token_type_ids

In [30]:
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model="bert-base-cased", freeze_bert=False):
        super(SentencePairClassifier, self).__init__()
        #  Instantiating BERT-based model object
        self.bert_layer = BertModel.from_pretrained(bert_model, return_dict=False)
        hidden_size = 768

        # Freeze bert layers and only train the classification layer weights
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False

        # Classification layer
        self.cls_layer = nn.Linear(hidden_size, 1)

        self.dropout = nn.Dropout(p=0.1)

    @autocast()  # run in mixed precision
    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to identify sentence1 and sentence2
        '''

        # Feeding the inputs to the BERT-based model to obtain contextualized representations
        # cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)
        cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)

        # Feeding to the classifier layer the last layer hidden-state of the [CLS] token further processed by a
        # Linear Layer and a Tanh activation. The Linear layer weights were trained from the sentence order prediction (ALBERT) or next sentence prediction (BERT)
        # objective during pre-training.
        logits = self.cls_layer(self.dropout(pooler_output))
        
        return logits

In [31]:
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate_loss(net, device, criterion, dataloader):
    net.eval()

    mean_loss = 0
    count = 0

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            count += 1

    return mean_loss / count

In [32]:
print("Creation of the models' folder...")
!mkdir models

Creation of the models' folder...


In [33]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
    iters = []
    train_losses = []
    val_losses = []

    scaler = GradScaler()

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                logits = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # Backpropagating the gradients
            # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, opti.step() is then called,
                # otherwise, opti.step() is skipped.
                scaler.step(opti)
                # Updates the scale for next iteration.
                scaler.update()
                # Adjust the learning rate based on the number of iterations.
                lr_scheduler.step()
                # Clear gradients
                opti.zero_grad()


            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0


        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1

    # Saving the model
    path_to_model='models/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(bert_model, lr, round(best_loss, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()

In [34]:
bert_model = "bert-base-cased"
freeze_bert = False  # if True, freeze the encoder weights and only update the classification layer weights
maxlen = 256
bs = 8
iters_to_accumulate = 2  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = 2e-5
epochs = 4

# Train and Evaluation

In [35]:
#  Set all seeds to make reproducible results
set_seed(1)

# Creating instances of training and validation set
print("Reading training data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
print("Reading validation data...")
val_set = CustomDataset(df_val, maxlen, bert_model)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=bs, num_workers=5,shuffle=True)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=5,shuffle=True)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert)

if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)

criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
num_warmup_steps = 0 # The number of steps for the warmup phase.
num_training_steps = epochs * len(train_loader)  # The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

Reading training data...


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Reading validation data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

...


Iteration 796/997 of epoch 4 complete. Loss : 0.05409316994490695 


100%|█████████▉| 996/997 [02:30<00:00,  6.49it/s]


Iteration 995/997 of epoch 4 complete. Loss : 0.05107294940776525 


100%|██████████| 333/333 [00:17<00:00, 19.38it/s]



Epoch 4 complete! Validation Loss : 0.11454431126425574
Best validation loss improved from 0.11458667387833467 to 0.11454431126425574

The model has been saved in models/bert-base-cased_lr_2e-05_val_loss_0.11454_ep_4.pt


# Predictions

In [36]:
def get_probs_from_logits(logits):
    """
    Converts a tensor of logits into an array of probabilities by applying the sigmoid function
    """
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()

def test_prediction(net, device, dataloader, with_labels=True, result_file="results/output.txt"):
    """
    Predict the probabilities on a dataset with or without labels and print the result in a file
    """
    net.eval()
    w = open(result_file, 'w')
    probs_all = []

    with torch.no_grad():
        if with_labels:
            for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
        else:
            for seq, attn_masks, token_type_ids in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()

    w.writelines(str(prob)+'\n' for prob in probs_all)
    w.close()

In [37]:
path_to_model = '/content/models/bert-base-cased_lr_2e-05_val_loss_0.11454_ep_4.pt'  

path_to_output_file = 'results/output.txt'

print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=5)

model = SentencePairClassifier(bert_model)
if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

print()
print("Loading the weights of the model...")
model.load_state_dict(torch.load(path_to_model))
model.to(device)

print("Predicting on test data...")
test_prediction(net=model, device=device, dataloader=test_loader, with_labels=True,  # set the with_labels parameter to False if your want to get predictions on a dataset without labels
                result_file=path_to_output_file)
print()
print("Predictions are available in : {}".format(path_to_output_file))

Reading test data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Loading the weights of the model...
Predicting on test data...


100%|██████████| 266/266 [00:13<00:00, 19.77it/s]


Predictions are available in : results/output.txt


# Evaluation

In [38]:
path_to_output_file = 'results/output.txt'  # path to the file with prediction probabilities

labels_test = df_test['label']  # true labels

probs_test = pd.read_csv(path_to_output_file, header=None)[0]  # prediction probabilities
threshold = 0.5   # you can adjust this threshold for your own dataset
preds_test=(probs_test>=threshold).astype('uint8') # predicted labels using the above fixed threshold

metric = load_metric("glue", "mrpc")

https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/tmp545k6392


Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

storing https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py in cache at /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py
creating metadata file for /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py
Checking /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py for additional imports.
Creating main folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/modules/datasets_modules/metrics/glue
Creating specific version folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/mod

In [39]:
# Compute the accuracy and F1 scores
metric._compute(predictions=preds_test, references=labels_test)

{'accuracy': 0.9463781749764817, 'f1': 0.96843853820598}

In [44]:
df_test.loc[2112]

sentence1        Trở ngại lớn nhất về khí hậu trong việc sử dụng tự nhiên của miền Bắc và Đông Bắc Bắc Bộ là:|nhịp điệu mùa khí hậu thất thường, thời tiết không ổn định|chịu tác động trực tiếp của gió mùa đông bắc, mùa đông lạnh|trong năm có hai mùa rõ rệt, thời tiết không ổn định|nhiều hiện tượng thời tiết cực đoan, thiên tai như bão, vòi rồng
sentence2                                                                                                                                 Thiên nhiên vùng núi Đông Bắc khác với Tây Bắc ở điểm:|mùa đông bớt lạnh nhưng khô hơn.|mùa hạ đến sớm, đôi khi có gió Tây.|mùa đông lạnh đến sớm và kết thúc muộn.|khí hậu lạnh chủ yếu do độ cao của địa hình.
ans_sentence1                                                                                                                                                                                                                                                                                   nhịp điệu mùa khí 